In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def simulate_schrodinger_1d(
    N: int = 1024,
    x_min: float = -20.0,
    x_max: float = 20.0,
    dt: float = 0.005,
    steps: int = 2500,
    save_every: int = 500,
    omega: float = 0.2,
    x0: float = -7.0,
    sigma: float = 1.0,
    k0: float = 2.0,
    potential_type: str = "harmonic",
    barrier_height: float = 1.5,
    barrier_width: float = 1.5,
):
    """
    Simulate a 1D wave packet under the time-dependent Schrodinger equation
    using a split-operator FFT method in natural units (hbar = m = 1).
    """
    x = np.linspace(x_min, x_max, N, endpoint=False)
    dx = x[1] - x[0]
    if potential_type == "harmonic":
        V = 0.5 * (omega**2) * (x**2)
    elif potential_type == "barrier":
        V = np.where(np.abs(x) < barrier_width / 2.0, barrier_height, 0.0)
    else:
        raise ValueError("potential_type must be 'harmonic' or 'barrier'")

    psi = np.exp(-((x - x0) ** 2) / (2 * sigma**2)) * np.exp(1j * k0 * x)

    def norm(wavefunc: np.ndarray) -> float:
        return float(np.sum(np.abs(wavefunc) ** 2) * dx)

    psi /= np.sqrt(norm(psi))

    k = 2 * np.pi * np.fft.fftfreq(N, d=dx)
    kinetic_phase = np.exp(-0.5j * (k**2) * dt)
    potential_half_phase = np.exp(-0.5j * V * dt)

    snapshots = [(0.0, np.abs(psi) ** 2)]
    for step in range(1, steps + 1):
        psi = potential_half_phase * psi
        psi_k = np.fft.fft(psi)
        psi_k = kinetic_phase * psi_k
        psi = np.fft.ifft(psi_k)
        psi = potential_half_phase * psi
        psi /= np.sqrt(norm(psi))

        if step % save_every == 0:
            snapshots.append((step * dt, np.abs(psi) ** 2))

    return x, V, psi, snapshots, norm(psi)


x, V, psi, snapshots, final_norm = simulate_schrodinger_1d()

max_density = max(np.max(density) for _, density in snapshots)
scaled_V = (V - np.min(V))
scaled_V = (scaled_V / np.max(scaled_V)) * (0.8 * max_density)

plt.figure(figsize=(10, 5))
plt.plot(x, scaled_V, "k--", linewidth=1.2, label="Scaled potential")
for t, density in snapshots:
    plt.plot(x, density, label=f"t = {t:.2f}")

plt.title("1D Time-Dependent Schrodinger Simulation")
plt.xlabel("x")
plt.ylabel(r"Probability density $|\psi(x,t)|^2$")
plt.legend(ncol=2)
plt.grid(alpha=0.25)
plt.tight_layout()
plt.show()

print(f"Final norm: {final_norm:.6f}")